In [8]:
import tensorflow as tf

# Wildfire Spread Prediction Feature Descriptions

| Feature Key  | Description |
|-------------|-------------|
| `vs`       | **Wind Speed** (m/s) – Measures how fast the wind is moving, affecting fire spread. |
| `th`       | **Theta (Potential Temperature)** – Represents the temperature an air parcel would have if moved adiabatically to a standard pressure level. |
| `population` | **Population Density** – Indicates how many people live in a given area, affecting fire risk and response strategies. |
| `tmmx`     | **Maximum Daily Temperature** (°C) – The highest temperature recorded during the day. |
| `PrevFireMask` | **Previous Fire Presence** (Binary/Mask) – Indicates if there was fire in the area the day before. |
| `elevation` | **Elevation** (meters) – The height above sea level, affecting weather conditions and fire behavior. |
| `sph`      | **Specific Humidity** (kg/kg) – The amount of water vapor per unit of air mass, influencing fuel moisture. |
| `pr`       | **Precipitation** (mm) – The amount of rainfall, which can suppress fire spread. |
| `pdsi`     | **Palmer Drought Severity Index (PDSI)** – A measure of drought conditions, with lower values indicating more severe drought. |
| `erc`      | **Energy Release Component (ERC)** – A fire weather index estimating the potential available energy in live and dead fuels. |
| `FireMask` | **Current Fire Presence** (Binary/Mask) – Indicates if there is an active fire in the area. |
| `NDVI`     | **Normalized Difference Vegetation Index (NDVI)** – A measure of vegetation health, where higher values indicate lush greenery and lower values indicate dry or dead vegetation. |
| `tmmn`     | **Minimum Daily Temperature** (°C) – The lowest temperature recorded during the day. |

---

### Why These Features Matter for Fire Spread Prediction
- **Temperature (`tmmx`, `tmmn`)**: Higher temperatures dry out vegetation, making it more flammable.
- **Humidity (`sph`)**: Lower humidity means drier conditions, increasing fire risk.
- **Wind Speed (`vs`)**: Stronger winds accelerate fire spread and can change fire direction unpredictably.
- **Precipitation (`pr`)**: Rainfall can suppress fire ignition and spread.
- **Drought Index (`pdsi`)**: Long-term dryness can make regions more susceptible to wildfires.
- **Energy Release Component (`erc`)**: Higher ERC values indicate more available fuel energy, leading to more intense fires.
- **Vegetation Health (`NDVI`)**: Green vegetation retains moisture, while dry vegetation is more flammable.
- **Elevation (`elevation`)**: Fires behave differently at higher altitudes due to changes in oxygen levels and terrain.
- **Fire Mask Features (`FireMask`, `PrevFireMask`)**: Indicate whether there were past or present wildfires in a given area, helping models track fire movement.

In [1]:
import tensorflow as tf
import os

# Define dataset file paths
tfrecord_dir = "data/"

def get_tfrecord_files(prefix):
    return sorted([os.path.join(tfrecord_dir, f) for f in os.listdir(tfrecord_dir) if f.startswith(prefix)])

# Automatically get all train, eval, and test files
train_tfrecord_files = get_tfrecord_files("next_day_wildfire_spread_train_")
eval_tfrecord_files = get_tfrecord_files("next_day_wildfire_spread_eval_")
test_tfrecord_files = get_tfrecord_files("next_day_wildfire_spread_test_")

# Load first record to check feature structure
def get_feature_names(tfrecord_files):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_files)
    for raw_record in raw_dataset.take(1):  # Take first record
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        return list(example.features.feature.keys())  # Extract feature names

feature_names = get_feature_names(train_tfrecord_files)
print("Feature Names:", feature_names)

def get_feature_schema(tfrecord_file):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
    for raw_record in raw_dataset.take(1):  # Inspect first record
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        schema = {}
        for key, feature in example.features.feature.items():
            dtype = feature.WhichOneof("kind")
            if dtype == "float_list":
                schema[key] = tf.io.FixedLenFeature([len(feature.float_list.value)], tf.float32)
            elif dtype == "int64_list":
                schema[key] = tf.io.FixedLenFeature([len(feature.int64_list.value)], tf.int64)
            elif dtype == "bytes_list":
                schema[key] = tf.io.FixedLenFeature([], tf.string)
        return schema

feature_schema = get_feature_schema(train_tfrecord_files[0])
print("Feature Schema:", feature_schema)

def parse_tfrecord_fn(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_schema)
    
    # Assuming each feature is a 64x64 grid, reshape accordingly
    cnn_input = tf.stack([
        tf.reshape(parsed_features['FireMask'], [64, 64]),
        tf.reshape(parsed_features['NDVI'], [64, 64]),
        tf.reshape(parsed_features['PrevFireMask'], [64, 64])
    ], axis=-1)  # Shape: (64, 64, 3)

    # No LSTM input needed for spatial data
    dense_input = tf.stack([parsed_features['vs'], parsed_features['tmmx'], parsed_features['tmmn']], axis=-1)
    
    # Reshape target to match model output
    target = tf.reduce_mean(parsed_features['FireMask'])  # Example: reduce to a single value

    return {'cnn_input': cnn_input, 'dense_input': dense_input}, target

def load_dataset(filenames, batch_size=32):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(parse_tfrecord_fn)
    dataset = dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load datasets
train_dataset = load_dataset(train_tfrecord_files)
eval_dataset = load_dataset(eval_tfrecord_files)
test_dataset = load_dataset(test_tfrecord_files)


Feature Names: ['NDVI', 'tmmn', 'elevation', 'population', 'FireMask', 'vs', 'pdsi', 'pr', 'tmmx', 'sph', 'th', 'PrevFireMask', 'erc']
Feature Schema: {'NDVI': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'tmmn': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'elevation': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'population': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'FireMask': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'vs': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'pdsi': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'pr': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'tmmx': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'sph': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'th': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=

2025-02-05 17:27:32.199029: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144


In [2]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Model

# Define CNN Model
cnn_input = Input(shape=(64, 64, 3), name="cnn_input")  # Adjust shape based on your data
x = Conv2D(32, (3, 3), activation="relu")(cnn_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation="relu")(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation="relu")(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=cnn_input, outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# Train the model
model.fit(train_dataset, epochs=20, batch_size=32)

Epoch 1/20


/Users/sydneybalboni/Documents/GitHub/fire-research/.venv/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: cnn_input
Received: inputs=['Tensor(shape=(None, 64, 64, 3))']
  warnings.warn(msg)


469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.0547 - loss: 0.0203
Epoch 2/20


2025-02-05 17:28:27.953746: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/sydneybalboni/Documents/GitHub/fire-research/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0562 - loss: 0.0133
Epoch 3/20


...

469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0564 - loss: 0.0136
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0582 - loss: 0.0129
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0554 - loss: 0.0137
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0574 - loss: 0.0122
Epoch 9/20


2025-02-05 17:29:40.326740: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0569 - loss: 0.0135
Epoch 10/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0553 - loss: 0.0140
Epoch 11/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0563 - loss: 0.0133
Epoch 12/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0540 - loss: 0.0138
Epoch 13/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.0557 - loss: 0.0136
Epoch 14/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.0538 - loss: 0.0140
Epoch 15/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0555 - loss: 0.0134
Epoch 16/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.0575 - loss: 0.0123
Epoch 17/20


2025-02-05 17:31:04.096483: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.0551 - loss: 0.0129
Epoch 18/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.0567 - loss: 0.0140
Epoch 19/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.0584 - loss: 0.0133
Epoch 20/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.0545 - loss: 0.0134


In [3]:
# Evaluate the model
eval_loss, eval_accuracy = model.evaluate(eval_dataset)
print(f"Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_accuracy}")

# Test the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0469 - loss: 0.0112
Evaluation Loss: 0.014339993707835674, Evaluation Accuracy: 0.0506126806139946
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0497 - loss: 0.0149
Test Loss: 0.016004350036382675, Test Accuracy: 0.05269390344619751
